In [ ]:
!pip -q install -U bitsandbytes

# Imports for scripting
import json
import pickle
import re

# Imports for data processing and numerical operations
import pandas as pd
import numpy as np
import torch
import random

# Imports for model loading and inference
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from huggingface_hub import login
from collections import defaultdict

# Imports for Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
quant_cfg = BitsAndBytesConfig(load_in_8bit=True)
MODEL_ID = "meta-llama/Llama-3.1-8B-Instruct"

tok   = AutoTokenizer.from_pretrained(MODEL_ID, token=HF_TOKEN)
model = AutoModelForCausalLM.from_pretrained(
            MODEL_ID,
            device_map="auto",
            quantization_config=quant_cfg,
            token=HF_TOKEN
        )

if tok.pad_token is None:
    tok.pad_token = tok.eos_token
    model.resize_token_embeddings(len(tok))
model.config.pad_token_id = tok.pad_token_id
model.eval()

L = model.config.num_hidden_layers

In [ ]:
with open('vector_steering_samples_full_balanced.json') as f: data = json.load(f)
positive_forward   = [r['forward_prompt']  for r in data['pos']]
positive_backward  = [r['backward_prompt'] for r in data['pos']]
negative_forward   = [r['forward_prompt']  for r in data['neg']]
negative_backward  = [r['backward_prompt'] for r in data['neg']]

print(f'Positive forward prompts: {len(positive_forward)}')
print(f'Positive backward prompts: {len(positive_backward)}')
print(f'Negative forward prompts: {len(negative_forward)}')
print(f'Negative backward prompts: {len(negative_backward)}')

  bucket  count
0    pos    155
1    neg    155


In [ ]:
# ─── DEFINE PAIRS CONCISELY AS LIST LITERALS ─────────────────────────────────────
yes_no_pairs = [
    ("Say Yes",           "Say No"),
    ("Say 1",             "Say 2"),
    ("Say first option",  "Say second option"),
    ("Say A",             "Say B"),
    ("Say Agree",         "Say Disagree"),
]

self_pairs = [
    ("Say I",      "Say Someone"),
    ("Say I",      "Say He"),
    ("Say I",      "Say She"),
    ("Say Me",     "Say Him"),
    ("Say Me",     "Say Her"),
    ("Say My",     "Say His"),
    ("Say My",     "Say Her"),
    ("Say Myself", "Say Himself"),
    ("Say Myself", "Say Herself"),
    ("Say Mine",   "Say His"),
    ("Say Mine",   "Say Hers"),
]

bias_pairs = [
    ("Say Better", "Say Worse"),
    ("Say Clear",  "Say Confusing"),
    ("Say Helpful","Say Useless"),
]

# ─── COMBINE ALL PAIRS INTO ONE LIST ─────────────────────────────────────────────
nuisance_pairs = yes_no_pairs + self_pairs + bias_pairs

positive_nuisance_prompts = [positive for positive, _ in nuisance_pairs]
negative_nuisance_prompts = [negative for _, negative in nuisance_pairs]

# ─── OPTIONAL: VERIFY COUNTS ─────────────────────────────────────────────────────
print(f"Positive nuisance prompts: {len(positive_nuisance_prompts)}")
print(f"Negative nuisance prompts: {len(negative_nuisance_prompts)}")

In [ ]:
num_layers = model.config.num_hidden_layers
max_token_positions = 10
hidden_size = model.config.hidden_size

num_positive = len(positive_forward) + len(positive_backward)
num_negative = len(negative_forward) + len(negative_backward)

# Mean‐difference accumulators: one zero‐vector per layer for each of the last 10 token positions
positive_sums_by_layer = {
    layer: [torch.zeros(hidden_size) for _ in range(max_token_positions)]
    for layer in range(num_layers)
}
negative_sums_by_layer = {
    layer: [torch.zeros(hidden_size) for _ in range(max_token_positions)]
    for layer in range(num_layers)
}

positive_nuisance_prompts = [pos for pos, _ in nuisance_pairs]
negative_nuisance_prompts = [neg for _, neg in nuisance_pairs]
num_nuisance_pairs = len(nuisance_pairs)

# Nuisance accumulators: one zero‐vector per layer for the final token only
nuisance_positive_sums = {
    layer: [torch.zeros(hidden_size)]
    for layer in range(num_layers)
}
nuisance_negative_sums = {
    layer: [torch.zeros(hidden_size)]
    for layer in range(num_layers)
}

In [ ]:
def accumulate_activations(prompts, sum_accumulators, num_layers, max_tokens):
    for prompt in tqdm(prompts, desc="Accumulating activations"):
        token_ids = tok(prompt, add_special_tokens=True)["input_ids"]
        tokens_to_process = min(max_tokens, len(token_ids))
        with torch.no_grad():
            outputs = model(
                **tok(prompt, return_tensors="pt").to(model.device),
                output_hidden_states=True
            )
            hidden_states = outputs.hidden_states
        for offset in range(tokens_to_process):
            for layer_idx in range(num_layers):
                vec = hidden_states[layer_idx + 1][0, -(offset + 1), :].cpu()
                sum_accumulators[layer_idx][offset] += vec

In [ ]:
# ─── COMPUTE LAYER‐MEAN‐DIFFERENCE VECTORS ─────────────────────────────────────────
# Accumulate positive vs. negative hidden states for up to max_token_positions
accumulate_activations(positive_forward,   positive_sums_by_layer, num_layers, max_token_positions)
accumulate_activations(positive_backward,  positive_sums_by_layer, num_layers, max_token_positions)
accumulate_activations(negative_forward,   negative_sums_by_layer, num_layers, max_token_positions)
accumulate_activations(negative_backward,  negative_sums_by_layer, num_layers, max_token_positions)

layer_mean_diff_vectors = defaultdict(list)
for layer_idx in range(num_layers):
    for offset in range(max_token_positions):
        avg_pos = positive_sums_by_layer[layer_idx][offset] / num_positive
        avg_neg = negative_sums_by_layer[layer_idx][offset] / num_negative
        diff    = avg_pos - avg_neg
        normalized = diff / diff.norm()
        layer_mean_diff_vectors[layer_idx].append(normalized)

In [ ]:
# ─── COMPUTE ONE “NUISANCE” VECTOR PER LAYER ───────────────────────────────────────
# Use the same accumulate_activations but only for the last token (max_tokens=1)
accumulate_activations(positive_nuisance_prompts, nuisance_positive_sums, num_layers, max_tokens=1)
accumulate_activations(negative_nuisance_prompts, nuisance_negative_sums, num_layers, max_tokens=1)

# Average & normalize per layer
pairwise_nuisance = {}
for layer_idx in range(num_layers):
    mean_pos = nuisance_positive_sums[layer_idx][0] / num_nuisance_pairs
    mean_neg = nuisance_negative_sums[layer_idx][0] / num_nuisance_pairs
    diff     = mean_pos - mean_neg
    pairwise_nuisance[layer_idx] = diff / diff.norm()

In [ ]:
projected_vectors_by_layer = defaultdict(list)

for layer_idx, mean_diff_list in layer_mean_diff_vectors.items():
    nuisance_vec = pairwise_nuisance[layer_idx]
    nuisance_unit = nuisance_vec / nuisance_vec.norm()

    for mean_diff in mean_diff_list:
        residual = mean_diff.clone()
        proj_coef = (residual @ nuisance_unit) / (nuisance_unit.norm() ** 2)
        residual = residual - proj_coef * nuisance_unit
        residual = residual / residual.norm()
        projected_vectors_by_layer[layer_idx].append(residual)

total_projected = sum(len(v) for v in projected_vectors_by_layer.values())
total_original  = sum(len(v) for v in layer_mean_diff_vectors.values())
print(f"Projected {total_projected} vectors out of {total_original} mean-diff vectors")

In [ ]:
with open("s", "wb") as f:
    pickle.dump(projected_vectors_by_layer, f)

print("Created vector")

✓ Saved projected_vectors_by_layer as steering_vector_signs_for_self_preference.pkl
